In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 25 19:32:35 2016

@author: SaiPhani and Carter
"""

# imports
from xlrd import open_workbook
from xlutils.copy import copy
import xlwt

import numpy as np
from scipy.interpolate import interp1d

import math

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, CheckButtons
from matplotlib.patches import Rectangle

# global variables
y_max = np.array([], dtype='float64')
x_max = np.array([], dtype='float64')
alpha = np.array([], dtype='float64')
maxEdges2 = np.array([])
maxEdges3 = np.array([])

#receive input/output file parameters
inputFile = input("Input file, e.g. input.txt : ")
smoothedOutput  = input("Output file for smoothed data, e.g. output.xls : ")
edgeOutput = input("Output file for detected edges, e.g. edges.txt : ")

# isolate x and y data
data = np.loadtxt(inputFile)
x = data[:,0]
y = data[:,1]

# receive window parameters from user and initialize thresholds
window_Span = float(input("Window size for algorithm 1, e.g. 0.025 : "))
window = float(input("Window size for algorithm 2 & 3, e.g. 0.025 : "))
print('')
total_Span  = 2 * window_Span
thresh = np.mean(np.absolute(np.diff(y))) * 3.5
prec = -10
while thresh / 10**(prec + 1) > 1:
    prec += 1
threshold1  = round(thresh, -prec)
threshold2  = round(thresh, -prec)
threshold3  = round(thresh, -prec)
toggle1 = True
toggle2 = True
toggle3 = True

for i in range(0, np.size(x)):
        alpha = np.append(alpha, [[np.mean(y[i]-y[(x >= x[i]-window_Span) & (x <= x[i]+window_Span)])]])                
        
def main():
    def smoothing_Plot(total_Span, threshold1):    
        global y_max, x_max
        x_max = np.array([])
        y_max = np.array([])
        nx_smooth1 = np.array([])
        nx_smooth2 = np.array([])
        nSmooth_Y_Coordinates = np.array([])
        
        nx_smooth1 = np.append(nx_smooth1, x[threshold1 < alpha])                
        
        for i in nx_smooth1:
           y_max = np.append(y_max, np.amax(y[(i-window_Span <= x) & (x <= i+window_Span)]))
        
        yset = set()
        newy_max = np.array([])
        for i in y_max:
            if i not in yset:
                newy_max = np.append(newy_max, i)
                yset.add(i)
        y_max = newy_max
        
        for i in y_max:
            x_max = np.append(x_max, x[np.where(y==i)[0][0]])
            
        newx_max = np.array([])
        i = 0
        while i < x_max.size:
            area = 1
            max = y_max[i]
            maxi = i
            while i + area < x_max.size and x_max[i + area] == x[np.where(x == x_max[i])[0][0] + area]:
                if y_max[i + area] > max:
                    max = y_max[i + area]
                    maxi = i + area
                area += 1
            newx_max = np.append(newx_max, x_max[maxi])
            i += area
        
        x_max = newx_max
        for i in x_max:
            nx_smooth2 = np.append(nx_smooth2, x[(i-window_Span<=x) & (x <= i+window_Span)])
        
        for i in x:
            if i in nx_smooth2:
                nSmooth_Y_Coordinates = np.append(nSmooth_Y_Coordinates, np.mean(y[(i-window_Span <= x) & (x <= i+window_Span)])) 
            else:
                nSmooth_Y_Coordinates = np.append(nSmooth_Y_Coordinates, y[np.where(x==i)])        
        return interp1d(x, nSmooth_Y_Coordinates, kind = "cubic")

    newx = x
    newy = np.zeros(newx.size)
    tolerance = np.mean(np.diff(newx)) / 100
    for i in range(0, newy.size):
        index = np.nonzero(x == newx[i])[0][0]
        min = index
        max = index
        while min > 0 and x[index] - x[min - 1] <= window + tolerance:
            min -= 1
        while max < y.size - 1 and x[max + 1] - x[index] <= window + tolerance:
            max += 1
        newy[i] = np.mean(y[min:max+1])

    workbook = xlwt.Workbook()
    sheet = workbook.add_sheet("Algo2 and Algo3") 
    sheet.write(0, 0, "(Raw-OB)/OB")
    sheet.write(0, 1, "wavelength (angs)")
    for i in range(len(newx)):        
        sheet.write(i+1, 0, newx[i])
        sheet.write(i+1, 1, newy[i])
    workbook.save(smoothedOutput)

    poly = interp1d(newx, newy, kind='cubic')
    xi = np.linspace(newx[0], newx[newx.size - 1], num=(newx[newx.size - 1] - newx[0])/.001, endpoint=True)

    ydiff = y - poly(newx)
    
    def calcEdges2(t):
        jumps = np.extract(ydiff > t, newx)
        maxEdges2 = np.array([])
        i = 0
        while i < jumps.size:
            area = 1
            max = ydiff[np.nonzero(newx == jumps[i])[0][0]]
            maxi = i
            while i + area < jumps.size and jumps[i + area] == x[np.nonzero(x == jumps[i])[0][0] + area]:
                if ydiff[np.nonzero(newx == jumps[i + area])[0][0]] > max:
                    max = ydiff[np.nonzero(newx == jumps[i + area])[0][0]]
                    maxi = i + area
                area += 1
            maxEdges2 = np.append(maxEdges2, jumps[maxi])
            i += area
        return maxEdges2

    def calcEdges3(t):
        jumps = np.extract(ydiff > t, newx)
        maxEdges3 = np.array([])
        i = 0
        while i < jumps.size:
            area = 1
            max = y[np.nonzero(x == jumps[i])[0][0]]
            maxi = i
            while i + area < jumps.size and jumps[i + area] == x[np.nonzero(x == jumps[i])[0][0] + area]:
                if y[np.nonzero(x == jumps[i + area])[0][0]] > max:
                    max = y[np.nonzero(x == jumps[i + area])[0][0]]
                    maxi = i + area
                area += 1
            maxEdges3 = np.append(maxEdges3, jumps[maxi])
            i += area
        return maxEdges3

    # plot with threshold values marked
    f = smoothing_Plot(window_Span, threshold1)    
    temp = f(x)    
    
    sheet = workbook.add_sheet("Algo1") 
    sheet.write(0, 0, "(Raw-OB)/OB")
    sheet.write(0, 1, "wavelength (angs)")
    for i in range(np.size(x)):        
        sheet.write(i+1, 0, temp[i])
        sheet.write(i+1, 1, x[i])
    workbook.save(smoothedOutput)  
    
    plt.close('all')    
    fig, ax = plt.subplots(6)        
    ori,    = ax[5].plot(x, y, '-o', color='Green', alpha = .85) # plot original points
    alg1,   = ax[5].plot(x, f(x), '-o', color='Purple', alpha = .75) # plot smoothed curve for algo1   
    alg2,   = ax[5].plot(newx, newy, 'o', color='Blue', alpha = .65) # plot smoothed curve for algo2
    cs,     = ax[5].plot(xi, poly(xi), color='Blue', alpha = .65) # plot cubic spline
        
    plt.xlabel("Wavelength (Angs)")
    plt.ylabel("(Raw - OB) / OB")
    plt.title("Neutron Imaging Graph")
    plt.gca().set_axis_bgcolor('White')
    ws1 = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
    ws2 = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
    plt.legend([ws1, ws2, ori, alg1, alg2], ("window size (1): " + str(window_Span), "window size (2 & 3): " + str(window), "original data", "algorithm 1", "algorithm 2 & 3"),  loc=3)
    
    global x_max, maxEdges2, maxEdges3 
    maxEdges2 = calcEdges2(threshold2)
    maxEdges3 = calcEdges3(threshold3)
    
    def drawEdgeLines():
        global threshold3, x_max, maxEdges2, maxEdges3, toggle1, toggle2, toggle3
        while len(ax[5].lines) > 4:
            ax[5].lines[-1].remove()
        if toggle1:
            for i in x_max:
                ax[5].axvline(x = i, linewidth=1.5, color='Orchid', alpha = .85)
        if toggle2:
            for i in range(0, maxEdges2.size):
                ax[5].axvline(x=maxEdges2[i], linewidth=1.5, color='Red', alpha = .75)
        if toggle3:
            for i in range(0, maxEdges3.size):
                ax[5].axvline(x=maxEdges3[i], linewidth=1.5, color='Gold', alpha = .65)
        plt.draw()
    drawEdgeLines()

    plt.subplots_adjust(bottom = 0.2)
    ax[0].set_position([0.15, 0.15, 0.70, 0.03])
    ax[1].set_position([0.15, 0.1, 0.70, 0.03])
    ax[2].set_position([0.15, 0.05, 0.70, 0.03])
    ax[3].set_position([0.88, 0.5, 0.08, 0.08])
    ax[4].set_position([0.88, 0.3, 0.08, 0.18])
    ax[5].set_position([0.15, 0.25, 0.70, 0.70])
    sthresh1 = Slider(ax[0], 'Threshold #1', 0.000, threshold1*3, valinit = threshold1, valfmt='%1.5f', color = 'Orchid', alpha = .75) 
    sthresh2 = Slider(ax[1], 'Threshold #2', 0.000, threshold2*3, valinit = threshold2, valfmt='%1.5f', color = 'Red', alpha = .75)
    sthresh3 = Slider(ax[2], 'Threshold #3', 0.000, threshold3*3, valinit = threshold3, valfmt='%1.5f', color = 'Gold', alpha = .75)
    bsave = Button(ax[3], 'Save\nEdges')
    checks = CheckButtons(ax[4], ('Show 1', 'Show 2', 'Show 3'), (True, True, True))

    def update1(val):
        global threshold1, x_max, maxEdges2, maxEdges3, toggle1
        if not toggle1:
            checks.set_active(0)
        threshold1 = sthresh1.val
        f1 = smoothing_Plot(total_Span, threshold1)
        temp = f1(x)    
        rb = open_workbook(smoothedOutput, formatting_info=True)
         
        wb = copy(rb) 
        w_sheet = wb.get_sheet(1) 
         
        w_sheet.write(0, 0, "(Raw-OB)/OB")
        w_sheet.write(0, 1, "wavelength (angs)")
        for i in range(np.size(x)):        
            w_sheet.write(i+1, 0, temp[i])
            w_sheet.write(i+1, 1, x[i])
        workbook.save(smoothedOutput)  
        fig.canvas.draw_idle()
        alg1.set_ydata(f1(x))
        drawEdgeLines()
    sthresh1.on_changed(update1)
    
    def update2(val):
        global threshold2, x_max, maxEdges2, maxEdges3, toggle2
        if not toggle2:
            checks.set_active(1)
        threshold2 = sthresh2.val
        maxEdges2 = calcEdges2(threshold2)
        fig.canvas.draw_idle()
        drawEdgeLines()
    sthresh2.on_changed(update2)
    
    def update3(val):
        global threshold3, x_max, maxEdges2, maxEdges3, toggle3
        if not toggle3:
            checks.set_active(2)
        threshold3 = sthresh3.val  
        maxEdges3 = calcEdges3(threshold3)
        fig.canvas.draw_idle()
        drawEdgeLines()
    sthresh3.on_changed(update3)
            
    def save(self):
        global threshold1, threshold2, threshold3, x_max, maxEdges2, maxEdges3
        print('Maximal points exceeding threshold {} for algo1: {}'.format(round(threshold1, 5), x_max))
        print('Maximal points exceeding threshold {} for algo2: {}'.format(round(threshold2, 5), maxEdges2))
        print('Maximal points exceeding threshold {} for algo3: {}\n'.format(round(threshold3, 5), maxEdges3))
        with open(edgeOutput, "a") as file:
            file.write('Maximal points exceeding threshold {} for algo1: {}\n'.format(round(threshold1, 5), x_max))
            file.write('Maximal points exceeding threshold {} for algo2: {}\n'.format(round(threshold2, 5), maxEdges2))
            file.write('Maximal points exceeding threshold {} for algo3: {}\n\n'.format(round(threshold3, 5), maxEdges3))
    bsave.on_clicked(save)
    
    def toggle(lbl):
        global toggle1, toggle2, toggle3
        if lbl == 'Show 1':
            toggle1 = not toggle1
        if lbl == 'Show 2':
            toggle2 = not toggle2
        if lbl == 'Show 3':
            toggle3 = not toggle3
        drawEdgeLines()
    checks.on_clicked(toggle)
    
    def hover(event):
        while (len(ax[5].texts) > 0):
            ax[5].texts[-1].remove()
        for i in range(4, len(ax[5].lines)):
            if ax[5].get_lines()[i].contains(event)[0]:
                tooltip = ax[5].text(event.xdata, event.ydata, ax[5].get_lines()[i].get_xdata()[0])
                tooltip.set_visible(True)
        plt.draw()
    fig.canvas.mpl_connect('motion_notify_event', hover)
    
    plt.show()
main()

Input file, e.g. input.txt : testin.txt
Output file for smoothed data, e.g. output.xls : output5.xls
Output file for detected edges, e.g. edges.txt : edges5.txt
Window size for algorithm 1, e.g. 0.025 : 0.025
Window size for algorithm 2 & 3, e.g. 0.025 : 0.025



Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Anaconda\envs\test\lib\tkinter\__init__.py", line 1550, in __call__
    return self.func(*args)
  File "C:\Anaconda\envs\test\lib\tkinter\__init__.py", line 596, in callit
    func(*args)
  File "C:\Anaconda\envs\test\lib\site-packages\matplotlib\backends\backend_tkagg.py", line 373, in idle_draw
    self.draw()
  File "C:\Anaconda\envs\test\lib\site-packages\matplotlib\backends\backend_tkagg.py", line 354, in draw
    FigureCanvasAgg.draw(self)
  File "C:\Anaconda\envs\test\lib\site-packages\matplotlib\backends\backend_agg.py", line 474, in draw
    self.figure.draw(self.renderer)
  File "C:\Anaconda\envs\test\lib\site-packages\matplotlib\artist.py", line 61, in draw_wrapper
    draw(artist, renderer, *args, **kwargs)
  File "C:\Anaconda\envs\test\lib\site-packages\matplotlib\figure.py", line 1159, in draw
    func(*args)
  File "C:\Anaconda\envs\test\lib\site-packages\matplotlib\artist.py", line 61, in draw_w

Maximal points exceeding threshold 0.003 for algo1: [ 1.83172881  2.19388106  3.26771009  3.83670533  4.04820012]
Maximal points exceeding threshold 0.003 for algo2: [ 1.83172881  2.21402248  3.27022777  3.83670533  4.04820012]
Maximal points exceeding threshold 0.003 for algo3: [ 1.83172881  2.21402248  3.26771009  3.83670533  4.04820012]

